In [1]:
import numpy as np
import pandas as pd
from utils import dictionaries

In [2]:
filename="../data/Eurostat Enterprise Stats Latest (inc.2023) (1).xlsx"

In [3]:
sheets=pd.ExcelFile(path_or_buffer=filename).sheet_names

In [4]:
def sheet_finder(sheet:str,filepath:str,n=8)->None:
    df=pd.read_excel(
        io=filepath,
        sheet_name=sheet,
        nrows=n
    )
    try:
        if ("Sheet" in sheet) and ("Enterprises - number" in df["Unnamed: 2"].values) and ("Total" not in df["Unnamed: 2"].values):
            return True
        else:
            return False
    except:
        return False

In [5]:
def sheet_reader(sheet:str,filepath:str)->None:
    df=pd.read_excel(
        io=filepath,
        sheet_name=sheet
    )
    segment=df.iloc[6]["Unnamed: 2"]
    columns=df.iloc[8].dropna()
    columns=["country"]+list(columns[columns!="TIME"].values)
    
    df=df.iloc[11:46].dropna(axis=1)
    df.columns=columns
    df["segment"]=segment
    df=df.apply(lambda col: col.replace(":",np.nan),axis=1)
    return df

In [6]:
sheet_flags=[]
for sheet in sheets:
    sheet_flags.append(sheet_finder(sheet=sheet,filepath=filename))

In [7]:
dfs=[]
for sheet,annotation in zip(sheets,sheet_flags):
    if annotation:
        dfs.append(sheet_reader(sheet=sheet,filepath=filename))

In [8]:
eurostat_data=pd.concat(dfs,axis=0)

In [9]:
eurostat_data["s_level_segment"]=eurostat_data["segment"].apply(
    lambda segment: dictionaries["eurostat_segments"].get(segment)
)

In [10]:
eurostat_data=(
    eurostat_data
    .groupby(by=["country","s_level_segment"])
    [["2021","2022","2023"]]
    .sum()
    .reset_index()
)

In [25]:
segment_pairs=[
    ("S5-S6","s5"),
    ("S5-S6","s6"),
    ("S7+","s7"),
    ("S7+","s8"),
    ("S7+","s9"),
    ("S7+","s10")
]

In [43]:
def map_eu_segments(
        df_to_map:pd.DataFrame,
        segment_pairs:list
    )->pd.DataFrame:
    dividable_segments=[segment[0] for segment in segment_pairs]
    cols_to_scale=["2021","2022","2023"]
    subdfs=[df_to_map[~df_to_map["s_level_segment"].isin(dividable_segments)]]
    for from_segment,to_segment in segment_pairs:
        subdf=df_to_map[df_to_map["s_level_segment"]==from_segment].copy(deep=True)
        subdf[cols_to_scale]=subdf[cols_to_scale]*dictionaries["segment_ratios"][to_segment]
        subdf["s_level_segment"]=to_segment
        subdfs.append(subdf)
    return pd.concat(subdfs)

In [48]:
non_eu=pd.read_excel(
    io="../data/Data Landscaping Master (Draft) - PQ WIP.xlsx",
    sheet_name="Non-Eurostat Hardcoded",
    skiprows=6
)

In [50]:
non_eu["s_level_segment"]=non_eu["Size class"].apply(
    lambda segment: dictionaries["eurostat_segments"].get(segment)
)
non_eu=non_eu.drop(columns=["Size class"])

In [55]:
non_eu=(
    non_eu
    .groupby(by=["Country","s_level_segment"])
    [["2021","2022","2023"]]
    .sum()
    .reset_index()
)

In [56]:
map_eu_segments(
    df_to_map=non_eu,
    segment_pairs=segment_pairs
)

,Country,s_level_segment,2021,2022,2023
0,Albania,S1-S3,9.702200e+04,1.061320e+05,1.160974e+05
1,Albania,S4,5.608000e+03,5.841000e+03,6.084400e+03
4,South Africa,S1-S3,1.780960e+05,1.798770e+05,1.816757e+05
5,South Africa,S4,6.917700e+04,6.986877e+04,7.056746e+04
8,Turkey,S1-S3,3.250625e+06,3.428611e+06,3.493503e+06
9,Turkey,S4,2.698170e+05,2.973170e+05,1.918050e+05
12,United Kingdom,S1-S3,2.480140e+06,2.476210e+06,2.428885e+06
13,United Kingdom,S4,2.328150e+05,2.390950e+05,2.442400e+05
2,Albania,s5,4.527628e+02,4.819138e+02,5.129417e+02
6,South Africa,s5,6.483327e+03,6.548160e+03,6.613642e+03


In [57]:
map_eu_segments(
    df_to_map=eurostat_data,
    segment_pairs=segment_pairs
)

,country,s_level_segment,2021,2022,2023
0,Albania,S1-S3,97022.000000,106132.000000,0.000000
1,Albania,S4,5608.000000,5841.000000,0.000000
4,Austria,S1-S3,543530.000000,535128.000000,532211.000000
5,Austria,S4,38752.000000,40986.000000,40804.000000
8,Belgium,S1-S3,808796.000000,853531.000000,841675.000000
...,...,...,...,...,...
123,Slovakia,s10,77.928289,79.013978,80.340930
127,Slovenia,s10,32.450015,33.656336,33.656336
131,Spain,s10,525.231846,564.195990,571.675176
135,Sweden,s10,204.230022,190.116075,199.766637
